<a href="https://colab.research.google.com/github/acafiero/sports-project2/blob/main/Cafiero_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, silhouette_score, classification_report, confusion_matrix

 ## *Dataset*
 1) github / Five Thirty Eight

 2) statistical data for modern NBA players

 3) What is the statiscal threshold of WAR and RAPTOR a player needs to reach to be worth signing

 4) Regression

 5) 21 features

 6) 4078 rows

 7) There are multiple entries for the same player that needed to be reduced and averaged out, several features that are redudant to the study and an excess of negative floats that could inaccurately affect the outcome

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
url = 'https://raw.githubusercontent.com/acafiero/sports-project2/main/modern_RAPTOR_by_player.csv'
df = pd.read_csv(url)
df.head()

,player_name,player_id,season,poss,mp,raptor_box_offense,raptor_box_defense,raptor_box_total,raptor_onoff_offense,raptor_onoff_defense,...,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
0,Alex Abrines,abrinal01,2017,2387,1135,0.745505,-0.372938,0.372567,-0.418553,-3.857011,...,0.543421,-1.144832,-0.601411,1.249008,1.447708,-0.198700,0.077102,-1.038677,-0.961575,0.326413
1,Alex Abrines,abrinal01,2018,2546,1244,0.317549,-1.725325,-1.407776,-1.291727,-0.049694,...,-0.020826,-1.502642,-1.523468,0.777304,0.465912,0.311392,-0.174621,-1.112625,-1.287247,-0.456141
2,Alex Abrines,abrinal01,2019,1279,588,-3.215683,1.078399,-2.137285,-6.158856,4.901168,...,-4.040157,1.885618,-2.154538,0.178167,0.178167,0.000000,-4.577678,1.543282,-3.034396,-0.268013
3,Precious Achiuwa,achiupr01,2021,1581,749,-4.122966,1.359278,-2.763688,-4.050779,-0.919712,...,-4.347596,0.954821,-3.392775,-0.246055,-0.246776,0.000721,-3.817713,0.474828,-3.342885,0.329157
4,Quincy Acy,acyqu01,2014,1716,847,-1.716079,0.133115,-1.582964,-0.324811,-1.661495,...,-1.565257,-0.216477,-1.781734,0.415918,0.415918,0.000000,-1.464417,-0.223754,-1.688171,-0.554898


In [4]:
#create a loop
#filter by player
#for each players WAR create new feature "War for next year"
#concatenate all together

In [5]:
dfml = pd.DataFrame()
players = df['player_name'].unique()

In [6]:
players

for player in players:
  player_df = df.loc[df['player_name'] == player]
  player_df['Next WAR'] = player_df['war_total'].shift(-1)
  dfml = pd.concat([dfml, player_df])

dfml.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,player_name,player_id,season,poss,mp,raptor_box_offense,raptor_box_defense,raptor_box_total,raptor_onoff_offense,raptor_onoff_defense,...,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact,Next WAR
0,Alex Abrines,abrinal01,2017,2387,1135,0.745505,-0.372938,0.372567,-0.418553,-3.857011,...,-1.144832,-0.601411,1.249008,1.447708,-0.198700,0.077102,-1.038677,-0.961575,0.326413,0.777304
1,Alex Abrines,abrinal01,2018,2546,1244,0.317549,-1.725325,-1.407776,-1.291727,-0.049694,...,-1.502642,-1.523468,0.777304,0.465912,0.311392,-0.174621,-1.112625,-1.287247,-0.456141,0.178167
2,Alex Abrines,abrinal01,2019,1279,588,-3.215683,1.078399,-2.137285,-6.158856,4.901168,...,1.885618,-2.154538,0.178167,0.178167,0.000000,-4.577678,1.543282,-3.034396,-0.268013,NaN
3,Precious Achiuwa,achiupr01,2021,1581,749,-4.122966,1.359278,-2.763688,-4.050779,-0.919712,...,0.954821,-3.392775,-0.246055,-0.246776,0.000721,-3.817713,0.474828,-3.342885,0.329157,NaN
4,Quincy Acy,acyqu01,2014,1716,847,-1.716079,0.133115,-1.582964,-0.324811,-1.661495,...,-0.216477,-1.781734,0.415918,0.415918,0.000000,-1.464417,-0.223754,-1.688171,-0.554898,-0.083313


In [7]:
dfml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4079 entries, 0 to 4078
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   player_name           4079 non-null   object 
 1   player_id             4079 non-null   object 
 2   season                4079 non-null   int64  
 3   poss                  4079 non-null   int64  
 4   mp                    4079 non-null   int64  
 5   raptor_box_offense    4078 non-null   float64
 6   raptor_box_defense    4078 non-null   float64
 7   raptor_box_total      4078 non-null   float64
 8   raptor_onoff_offense  4078 non-null   float64
 9   raptor_onoff_defense  4078 non-null   float64
 10  raptor_onoff_total    4078 non-null   float64
 11  raptor_offense        4079 non-null   float64
 12  raptor_defense        4079 non-null   float64
 13  raptor_total          4079 non-null   float64
 14  war_total             4079 non-null   float64
 15  war_reg_season       

In [8]:
dfml.duplicated().sum()

## no duplicated values

0

In [9]:
dfml.nunique()

## checking all the various features

player_name             1192
player_id               1187
season                     8
poss                    2867
mp                      2130
raptor_box_offense      4078
raptor_box_defense      4078
raptor_box_total        4078
raptor_onoff_offense    4077
raptor_onoff_defense    4077
raptor_onoff_total      4077
raptor_offense          4079
raptor_defense          4079
raptor_total            4079
war_total               4079
war_reg_season          4075
war_playoffs            1717
predator_offense        4079
predator_defense        4079
predator_total          4079
pace_impact             4078
Next WAR                2887
dtype: int64

In [10]:
dfml.isna().sum()

## checking for rows missing data

player_name                0
player_id                  0
season                     0
poss                       0
mp                         0
raptor_box_offense         1
raptor_box_defense         1
raptor_box_total           1
raptor_onoff_offense       1
raptor_onoff_defense       1
raptor_onoff_total         1
raptor_offense             0
raptor_defense             0
raptor_total               0
war_total                  0
war_reg_season             0
war_playoffs               0
predator_offense           0
predator_defense           0
predator_total             0
pace_impact                1
Next WAR                1192
dtype: int64

In [11]:
dfml.dropna()
dfml.isna().sum()

## dropped all rows missing values from the dataframe

player_name                0
player_id                  0
season                     0
poss                       0
mp                         0
raptor_box_offense         1
raptor_box_defense         1
raptor_box_total           1
raptor_onoff_offense       1
raptor_onoff_defense       1
raptor_onoff_total         1
raptor_offense             0
raptor_defense             0
raptor_total               0
war_total                  0
war_reg_season             0
war_playoffs               0
predator_offense           0
predator_defense           0
predator_total             0
pace_impact                1
Next WAR                1192
dtype: int64

In [12]:
dfml.drop(['player_name', 'player_id', 'season', 'raptor_box_offense', 'raptor_box_defense', 'raptor_onoff_offense', 'raptor_onoff_defense', 'raptor_offense', 'raptor_defense', 'raptor_box_total', 'war_reg_season', 'war_playoffs', 'predator_offense', 'predator_defense'], axis=1, inplace=True)
dfml.head()

,poss,mp,raptor_onoff_total,raptor_total,war_total,predator_total,pace_impact,Next WAR
0,2387,1135,-4.275564,-0.601411,1.249008,-0.961575,0.326413,0.777304
1,2546,1244,-1.341421,-1.523468,0.777304,-1.287247,-0.456141,0.178167
2,1279,588,-1.257688,-2.154538,0.178167,-3.034396,-0.268013,NaN
3,1581,749,-4.970491,-3.392775,-0.246055,-3.342885,0.329157,NaN
4,1716,847,-1.986306,-1.781734,0.415918,-1.688171,-0.554898,-0.083313


In [13]:
dfml = dfml.loc[dfml['war_total'] >= 1]
dfml = dfml.loc[dfml['predator_total'] >= 1]
dfml = dfml.loc[dfml['raptor_total'] >= 1]
dfml = dfml.loc[dfml['pace_impact'] >= 1]
dfml = dfml.loc[dfml['Next WAR'] >= 1]
dfml

,poss,mp,raptor_onoff_total,raptor_total,war_total,predator_total,pace_impact,Next WAR
127,6265,2872,5.543546,6.806518,14.269861,7.083689,1.319042,13.149613
128,4894,2194,9.090994,8.778267,13.149613,7.521699,1.799512,13.666796
160,6225,3130,2.318868,2.146961,8.034787,2.400297,1.319376,7.993878
272,6871,3415,2.518748,2.140857,8.669646,3.200148,1.108855,4.220007
309,6485,3028,1.107369,3.374096,9.580186,3.724767,1.283573,4.656472
...,...,...,...,...,...,...,...,...
3875,6533,3149,8.175709,4.429313,11.841404,5.656411,2.941962,6.142006
3876,6169,2827,0.451285,1.431787,6.142006,2.696536,2.660690,2.127703
3948,4821,2266,3.572633,3.677213,7.619304,2.484788,1.523881,6.274484
3949,5489,2589,1.722066,1.882814,6.274484,1.759520,2.511937,4.286623


In [14]:
dfml['Next WAR'].fillna(value=df['Next WAR'].mean())
dfml

KeyError: ignored

In [ ]:
dfml.describe().round(2)

In [ ]:
corr = dfml.corr()
sns.heatmap(corr, cmap = 'coolwarm', annot = True)

## we can see WAR, PREDATOR, total RAPTOR and Pace Impact have a fairly close correlations

In [ ]:
sns.scatterplot(data=dfml, x='war_total', y='mp', legend = False)

In [ ]:
sns.jointplot(data=dfml, x='war_total', y='pace_impact', legend = False)

In [ ]:
#@title
sns.scatterplot(data=df, x='war_total', y='raptor_total', legend = False)

## there are several distinct outliers that give us a better picture of who could be our ideal player

In [ ]:
maxWAR = dfml[['war_total', 'raptor_total']].max()
print(maxWAR) 

In [ ]:
print('Our highest WAR candidate is ')
print(dfml[dfml.war_total == dfml.war_total.max()])

In [ ]:
print('Our highest RAPTOR candidate is ')
print(dfml[dfml.raptor_total == dfml.raptor_total.max()])

## our top choice is averages 26.6 WAR, almost a 12.5 on RAPTOR

In [ ]:
x = dfml[['war_total', 'raptor_total']]
x

In [ ]:
scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)

In [ ]:
ks = range(2,9) 
inertias = []

for k in ks:
   kmeans = KMeans(n_clusters=k, random_state=42)
   kmeans.fit(scaled_x)
   inertias.append(kmeans.inertia_)

plt.plot(ks, inertias, marker = '.')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
ks = range(2,8)
sil = []

for k in ks:
  kmeans = KMeans(n_clusters=k, random_state=42)
  kmeans.fit(scaled_x)
  sil.append(silhouette_score(scaled_x, kmeans.labels_))

plt.plot(ks, sil, marker = '.')
plt.xlabel('Number of Clusters')
plt.ylabel('Silo Score')
plt.show()

In [ ]:
km = KMeans(n_clusters=2, random_state=42)
km.fit(scaled_x)
silhouette_score(scaled_x, kmeans.labels_)

In [ ]:
kmeans3 = KMeans(n_clusters = 3, random_state = 42)
kmeans3.fit(scaled_x)

In [ ]:
dfml['cluster3'] = kmeans3.labels_

plt.scatter(dfml['war_total'], dfml['raptor_total'], c = dfml['cluster3'])
plt.xlabel("Wins Above Replacement")
plt.ylabel("RAPTOR")
plt.show()

In [ ]:
cluster_groups = dfml.groupby('cluster3', as_index=False).mean()
cluster_groups

In [ ]:
fig, axes = plt.subplots(2,5, figsize = (20, 10))

axes = axes.ravel()

for i, col in enumerate(cluster_groups.columns[1:]):
  axes[i].bar(cluster_groups['cluster3'], cluster_groups[col])
  axes[i].set_title(f'Mean {col}')

In [ ]:
y = dfml['war_total']
X = dfml.drop(columns = 'war_total')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
pca = PCA(n_components = .98)
knn = KNeighborsRegressor()

preprocessor = make_pipeline(scaler, pca)

In [ ]:
knn_pca_pipe = make_pipeline(preprocessor, knn)
knn_pca_pipe.fit(X_train, y_train)

In [ ]:
def evaluate_model(y_true, y_pred):
  
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  r2 = r2_score(y_true, y_pred)
  rmse = np.sqrt(mse)
  print(f'MAE {mae}\n MSE {mse}\n R^2 {r2}\n RMSE{rmse}')

In [ ]:
knn_pca_pipe_trainpreds = knn_pca_pipe.predict(X_train)
knn_pca_pipe_testpreds = knn_pca_pipe.predict(X_test)

In [ ]:
print('Training Data')
evaluate_model(y_train, knn_pca_pipe_trainpreds)
print('\n')
print('Test Data')
evaluate_model(y_test, knn_pca_pipe_testpreds)

In [ ]:
knn_pipe = make_pipeline(scaler, KNeighborsRegressor())
knn_pipe.fit(X_train, y_train)

In [ ]:
pred_no_pca = knn_pipe.predict(X_test)
pred_no_pca

In [ ]:
acc_score = knn_pipe.score(X_test, y_test)
acc_score

In [ ]:
d_tree = DecisionTreeRegressor(random_state = 42)
d_tree.fit(X_train, y_train)

In [ ]:
dtrain_pred = d_tree.predict(X_train)
dtest_pred = d_tree.predict(X_test)

In [ ]:
print('Training Data')
evaluate_model(y_train, dtrain_pred)
print('\n')
print('Test Data')
evaluate_model(y_test, dtest_pred )

In [ ]:
dtrain_score = d_tree.score(X_train, y_train)
dtest_score = d_tree.score(X_test, y_test)
print(dtrain_score)
print(dtest_score)

In [ ]:
d_tree.get_depth()

In [ ]:
d_tree_12 = DecisionTreeRegressor(max_depth = 11, random_state = 42)
d_tree_12.fit(X_train, y_train)
dtrain_12_score = d_tree_12.score(X_train, y_train)
dtest_12_score = d_tree_12.score(X_test, y_test)
print(dtrain_12_score)
print(dtest_12_score)

In [ ]:
d_tree_9 = DecisionTreeRegressor(max_depth = 9, random_state = 42)
d_tree_9.fit(X_train, y_train)
dtrain_9_score = d_tree_9.score(X_train, y_train)
dtest_9_score = d_tree_9.score(X_test, y_test)
print(dtrain_9_score)
print(dtest_9_score)

In [ ]:
d_tree_5 = DecisionTreeRegressor(max_depth = 5, random_state = 42)
d_tree_5.fit(X_train, y_train)
dtrain_5_score = d_tree_5.score(X_train, y_train)
dtest_5_score = d_tree_5.score(X_test, y_test)
print(dtrain_5_score)
print(dtest_5_score)

## there is an approximate difference of .0711 between our training and testing data

## In comparison to the PCA and KNN without PCA, it can be seen that a Decision Tree Regression yields the least amount of overfitting.